**Download Huggingface Dataset Handler**

In [14]:
!pip install datasets

**Imports**

In [15]:
import PIL
import pandas as pd
from PIL import Image
from matplotlib import pyplot as plt
from scipy.spatial import distance
import numpy as np

import cv2
from google.colab.patches import cv2_imshow
import matplotlib.image as pltimg
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler

**Download CIFAR 100 TEST dat**a

In [16]:
from datasets import load_dataset
dataset = load_dataset('cifar100', split='test')

Reusing dataset cifar100 (/root/.cache/huggingface/datasets/cifar100/cifar100/1.0.0/0f9be8dd0480d385177a5c250878f4480651bbf0fc86d714b33d56c9aaad5160)


# **RGB Computations**

**Allocate locations of images from test data**

In [17]:
'''
Index - Class from data: 
0 Apple ,
53 Orange, 
22 Clock,
telephone 86, 
television 87 
Spider 79, 
man 46, 
lobster 45, 
boy 11
'''
# Store the locations of the images in lists
apple_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 0]
orange_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 53]
clock_locs = [i for i,v in enumerate(dataset['fine_label']) if v ==22]
telephone_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 86]
television_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 87]
spider_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 79]
man_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 46]
lobster_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 45]
boy_locs = [i for i,v in enumerate(dataset['fine_label']) if v == 11]


In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
def savesamples(itemloclist,name):
  for loc in itemloclist:
    PIL_image = Image.fromarray(np.array(dataset['img'][loc]).astype('uint8'), 'RGB').resize((64,64))
    PIL_image.save('/content/drive/MyDrive/Machine Learning/Zero Shot Learning/NTECH/Image Data/CIFAR100_Sample/{}_{}.png'.format(name,loc))

In [20]:
# savesamples(apple_locs,'Apple')
# savesamples(orange_locs, 'Orange') 
# savesamples(clock_locs , 'Clock')
# savesamples(telephone_locs ,'Telephone')
# savesamples(television_locs , 'Television')
# savesamples(spider_locs , 'Spider')
# savesamples(man_locs , 'Man')
# savesamples(lobster_locs , 'Lobster')
savesamples(boy_locs, 'Boy')

**Utility functions for RGB - Cosine Similarity Computations**

In [ ]:
# Compute mean RGB from the images
def get_mean_rgb(itemloclist):
    means = [] # Store means of each images
    # Load images from list, calculate width height, get RGB colors.
    for loc in itemloclist:
        PIL_image = Image.fromarray(np.array(dataset['img'][loc]).astype('uint8'), 'RGB')#.resize((128,128))
        w,h = PIL_image.size
        colors = PIL_image.getcolors(w*h)
        mean = [0, 0, 0]
        counter = 0
        # result = sorted(colors, key=lambda tup: tup[0], reverse=True)
        
        # From each image, add all Rs, Gs, Bs to list. At the end of loading compute mean for that image and append to means.
        for res in colors:
            #red = res[0][0]
            mean[0] += res[1][0]
            #green = res[0][1]
            mean[1] += res[1][1]
            #blue = res[0][2]
            mean[2] += res[1][2]
            counter +=1
        means.append([x/(counter+0.0001) for x in mean]) # +0.0001 to avoid division by 0
    return means


# Compute euclidean distances of mean RGB to target colors: Red, Green, Blue, Orange, and Black
def get_euc_dist(meanslist):
  euc_dist_r = []     # Euclidean Distance of Sample to Red Color
  euc_dist_g = []     # Euclidean Distance of Sample to Green Color
  euc_dist_b = []     # Euclidean Distance of Sample to Blue Color
  euc_dist_o = []     # Euclidean Distance of Sample to Orange Color
  euc_dist_bl = []    # Euclidean Distance of Sample to Black Color
  for datapoints in meanslist:
    xs = datapoints[0]
    ys = datapoints[1] 
    zs = datapoints[2]
    euc_dist_r.append(distance.euclidean([xs,ys,zs], [255, 0, 0]))
    euc_dist_g.append(distance.euclidean([xs,ys,zs], [0, 255, 0]))
    euc_dist_b.append(distance.euclidean([xs,ys,zs], [0, 0, 255]))
    euc_dist_o.append(distance.euclidean([xs,ys,zs], [255, 165, 0]))
    euc_dist_bl.append(distance.euclidean([xs,ys,zs], [0, 0, 0]))
  return {"euc_dist_r":euc_dist_r, 
  "euc_dist_g":euc_dist_g,
  "euc_dist_b":euc_dist_b,
  "euc_dist_o":euc_dist_o,
  "euc_dist_bl":euc_dist_bl}

# Calculate the overall mean of the euclidean distances.
def get_mean_colors_from_all_images(dists):
  return np.array(normalizeList(dists['euc_dist_r'])).mean(), np.array(normalizeList(dists['euc_dist_g'])).mean() ,np.array(normalizeList(dists['euc_dist_b'])).mean(),np.array(normalizeList(dists['euc_dist_o'])).mean(),np.array(normalizeList(dists['euc_dist_bl'])).mean()


In [ ]:
def get_mean_colors_from_all_images(dists):
  return np.array(normalizeList(dists['euc_dist_r'])).mean(), np.array(normalizeList(dists['euc_dist_g'])).mean() ,np.array(normalizeList(dists['euc_dist_b'])).mean(),np.array(normalizeList(dists['euc_dist_o'])).mean(),np.array(normalizeList(dists['euc_dist_bl'])).mean()


In [ ]:
get_mean_colors_from_all_images(boy_dists)

(0.7807761740153684,
 0.8135194248151673,
 0.7941322520904851,
 0.790675549283638,
 0.6328791698158537)

**Compute Mean RGBs, Euclidean Distances, Mean Euclidean Distances**

In [ ]:
# For Apple
apple_means = get_mean_rgb(apple_locs)
apple_dists = get_euc_dist(apple_means)
apple_col_means = get_mean_colors_from_all_images(apple_dists)
print('Apple Done')
# For Orange
orange_means = get_mean_rgb(orange_locs)
orange_dists = get_euc_dist(orange_means)
orange_col_means = get_mean_colors_from_all_images(orange_dists)
print('Orange Done')
# For Clock
clock_means = get_mean_rgb(clock_locs)
clock_dists = get_euc_dist(clock_means)
clock_col_means = get_mean_colors_from_all_images(clock_dists)
print('Clock Done')
# For Telephone
telephone_means = get_mean_rgb(telephone_locs)
telephone_dists = get_euc_dist(telephone_means)
telephone_col_means = get_mean_colors_from_all_images(telephone_dists)
print('Phone Done')
# For Television
television_means = get_mean_rgb(television_locs)
television_dists = get_euc_dist(television_means)
television_col_means = get_mean_colors_from_all_images(television_dists)
print('TV Done')
# For Spider
spider_means = get_mean_rgb(spider_locs)
spider_dists = get_euc_dist(spider_means)
spider_col_means = get_mean_colors_from_all_images(spider_dists)
print('Spider Done')
# For Man
man_means = get_mean_rgb(man_locs)
man_dists = get_euc_dist(man_means)
man_col_means = get_mean_colors_from_all_images(man_dists)
print('Man Done')
# For Lobster
lobster_means = get_mean_rgb(lobster_locs)
lobster_dists = get_euc_dist(lobster_means)
lobster_col_means = get_mean_colors_from_all_images(lobster_dists)
print('Lobster Done')
# For Boy
boy_means = get_mean_rgb(boy_locs)
boy_dists = get_euc_dist(boy_means)
boy_col_means = get_mean_colors_from_all_images(boy_dists)
print('Boy Done')

Apple Done
Orange Done
Clock Done
Phone Done
TV Done
Spider Done
Man Done
Lobster Done
Boy Done


In [ ]:
l =[0,3,4]
[float(i)/max(l) for i in l]

[0.0, 0.75, 1.0]

In [ ]:
def normalizeList(l):
  return [float(i)/max(l) for i in l]

In [ ]:
normalizeList(apple_col_means)

[0.712653838807539,
 0.9191385466697349,
 1.0,
 0.6040753006888596,
 0.832639601472936]

**Converting Result into CSV**

In [ ]:
def conv2df(means, rd, gd, bd, od,bld,label):
  cls = [label]*len(rd)
  col_data = {'Mean RGB':means,
            'Red':rd,
            'Green':gd,
            'Blue':bd,
            'Orange':od,
            'Black':bld,
            'Class':cls
            }
  return pd.DataFrame.from_dict(col_data)

In [ ]:
apple_df = conv2df(apple_means,
                   normalizeList(apple_dists['euc_dist_r']),
                   normalizeList(apple_dists['euc_dist_g']),
                   normalizeList(apple_dists['euc_dist_b']),
                   normalizeList(apple_dists['euc_dist_o']),
                   normalizeList(apple_dists['euc_dist_bl']),
                   'Apple')
orange_df = conv2df(orange_means,
                   normalizeList(orange_dists['euc_dist_r']),
                   normalizeList(orange_dists['euc_dist_g']),
                   normalizeList(orange_dists['euc_dist_b']),
                   normalizeList(orange_dists['euc_dist_o']),
                   normalizeList(orange_dists['euc_dist_bl']),
                   'Orange')
clock_df = conv2df(clock_means,
                   normalizeList(clock_dists['euc_dist_r']),
                   normalizeList(clock_dists['euc_dist_g']),
                   normalizeList(clock_dists['euc_dist_b']),
                   normalizeList(clock_dists['euc_dist_o']),
                   normalizeList(clock_dists['euc_dist_bl']),
                   'Clock')
telephone_df = conv2df(telephone_means,
                   normalizeList(telephone_dists['euc_dist_r']),
                   normalizeList(telephone_dists['euc_dist_g']),
                   normalizeList(telephone_dists['euc_dist_b']),
                   normalizeList(telephone_dists['euc_dist_o']),
                   normalizeList(telephone_dists['euc_dist_bl']),
                   'Telephone')
television_df = conv2df(television_means,
                   normalizeList(television_dists['euc_dist_r']),
                   normalizeList(television_dists['euc_dist_g']),
                   normalizeList(television_dists['euc_dist_b']),
                   normalizeList(television_dists['euc_dist_o']),
                   normalizeList(television_dists['euc_dist_bl']),
                   'Television')
spider_df = conv2df(spider_means,
                   normalizeList(spider_dists['euc_dist_r']),
                   normalizeList(spider_dists['euc_dist_g']),
                   normalizeList(spider_dists['euc_dist_b']),
                   normalizeList(spider_dists['euc_dist_o']),
                   normalizeList(spider_dists['euc_dist_bl']),
                   'Spider')
man_df = conv2df(man_means,
                   normalizeList(man_dists['euc_dist_r']),
                   normalizeList(man_dists['euc_dist_g']),
                   normalizeList(man_dists['euc_dist_b']),
                   normalizeList(man_dists['euc_dist_o']),
                   normalizeList(man_dists['euc_dist_bl']),
                   'Man')
lobster_df = conv2df(lobster_means,
                   normalizeList(lobster_dists['euc_dist_r']),
                   normalizeList(lobster_dists['euc_dist_g']),
                   normalizeList(lobster_dists['euc_dist_b']),
                   normalizeList(lobster_dists['euc_dist_o']),
                   normalizeList(lobster_dists['euc_dist_bl']),
                   'Lobster')
boy_df = conv2df(boy_means,
                   normalizeList(boy_dists['euc_dist_r']),
                   normalizeList(boy_dists['euc_dist_g']),
                   normalizeList(boy_dists['euc_dist_b']),
                   normalizeList(boy_dists['euc_dist_o']),
                   normalizeList(boy_dists['euc_dist_bl']),
                   'Boy')

In [ ]:
concat_df = pd.concat([apple_df,orange_df,clock_df,telephone_df,television_df,spider_df,man_df,lobster_df,boy_df]).reset_index(drop = True)

In [ ]:
concat_df.to_csv('CIFAR100_COLOR_DIST_NEW_cosine.csv',index=False)

# **Edge Computations**

In [ ]:
# Connect to drive to fetch shapes: Circle, Square/Rectangle
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Get edges of Target shapes: Circle, Square/Rectangle**

In [ ]:
def getEdgeofShapes(pathToShape):
  img = cv2.imread(pathToShape)
  img = cv2.resize(img, (32, 32))
  img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  edges = cv2.Canny(image=img_gray, threshold1=100, threshold2=200)
  return edges

# Detect edge of circle
edges_circle = getEdgeofShapes('/content/drive/MyDrive/Machine Learning/Zero Shot Learning/NTECH/Image Data/Shapes/circle.png')
# Detect edge of rectangle / square
edges_square = getEdgeofShapes('/content/drive/MyDrive/Machine Learning/Zero Shot Learning/NTECH/Image Data/Shapes/square.png')

**Calculate Cosine Distances from image edges to target edges**

In [ ]:
def minkowskidist(locs):
  dist_circle = []
  dist_rect = []
  for loc in locs:
    PIL_image = Image.fromarray(np.array(dataset['img'][loc]).astype('uint8'), 'RGB')
    img_gray = cv2.cvtColor(np.float32(PIL_image), cv2.COLOR_BGR2GRAY)
    edges_apple = cv2.Canny(image=np.uint8(img_gray), threshold1=100, threshold2=200)
    
    dist_circle.append(1- distance.cosine(edges_circle.flatten(), edges_apple.flatten()))
    dist_rect.append(1- distance.cosine(edges_square.flatten(), edges_apple.flatten()))
  return {'circle':dist_circle, 'rect':dist_rect}

In [ ]:
apple_shape_dist = minkowskidist(apple_locs)
orange_shape_dist = minkowskidist(orange_locs)
clock_shape_dist = minkowskidist(clock_locs)
telephone_shape_dist = minkowskidist(telephone_locs)
television_shape_dist = minkowskidist(television_locs)
spider_shape_dist = minkowskidist(spider_locs)
man_shape_dist = minkowskidist(man_locs)
lobster_shape_dist = minkowskidist(lobster_locs)
boy_shape_dist = minkowskidist(boy_locs)

**Convert them into Dataframes to save CSV**

In [ ]:
len(apple_shape_dist_df)

100

In [ ]:
apple_shape_dist_df = pd.DataFrame.from_dict(apple_shape_dist)
orange_shape_dist_df = pd.DataFrame.from_dict(orange_shape_dist)
clock_shape_dist_df = pd.DataFrame.from_dict(clock_shape_dist)
telephone_shape_dist_df = pd.DataFrame.from_dict(telephone_shape_dist)
television_shape_dist_df = pd.DataFrame.from_dict(television_shape_dist)
spider_shape_dist_df = pd.DataFrame.from_dict(spider_shape_dist)
man_shape_dist_df = pd.DataFrame.from_dict(man_shape_dist)
lobster_shape_dist_df = pd.DataFrame.from_dict(lobster_shape_dist)
boy_shape_dist_df = pd.DataFrame.from_dict(boy_shape_dist)

name_col = ['Apple']*100 + ['Orange']*100 + ['Clock']*100 + ['Telephone']*100 + ['Television']*100 + ['Spider']*100 + ['Man']*100 + ['Lobster']*100 + ['Boy']*100

shape_df = pd.concat([apple_shape_dist_df,orange_shape_dist_df,clock_shape_dist_df,telephone_shape_dist_df,television_shape_dist_df,spider_shape_dist_df,man_shape_dist_df,lobster_shape_dist_df,boy_shape_dist_df]).reset_index(drop=True)
shape_df['class'] = name_col
shape_df.to_csv('Cosine Shape Dist.csv',index=False)

In [ ]:
print(np.array(apple_shape_dist['circle']).mean(), np.array(apple_shape_dist['rect']).mean())
print(np.array(orange_shape_dist['circle']).mean(), np.array(orange_shape_dist['rect']).mean())
print(np.array(clock_shape_dist['circle']).mean(), np.array(clock_shape_dist['rect']).mean())
print(np.array(telephone_shape_dist['circle']).mean(), np.array(telephone_shape_dist['rect']).mean())
print(np.array(television_shape_dist['circle']).mean(), np.array(television_shape_dist['rect']).mean())
print(np.array(spider_shape_dist['circle']).mean(), np.array(spider_shape_dist['rect']).mean())
print(np.array(man_shape_dist['circle']).mean(), np.array(man_shape_dist['rect']).mean())
print(np.array(lobster_shape_dist['circle']).mean(), np.array(lobster_shape_dist['rect']).mean())
print(np.array(boy_shape_dist['circle']).mean(), np.array(boy_shape_dist['rect']).mean())

0.1697301241362723 0.13803807024906073
0.15074805196534014 0.12871741698131164
0.1342179495721808 0.13096857774895773
0.1252142658949014 0.10931388571428688
0.13073433643669852 0.13547626302709853
0.08940934134122122 0.08739553210847933
0.11704146315031076 0.11760031046369603
0.14132818912360243 0.12979228441186438
0.12416545127769356 0.12775947053774198


# **Semantic Distance Computations**

**Semantic Vector Computations**

In [ ]:
""" 
###Classes###         
Apple
Orange
Clock
Telephone
Television
Spider
Man
Lobster
Boy

###Semantic Combinations###
Apple - Orange
Apple - Clock
Apple - Telephone
Apple - Television
Apple - Spider
Apple - Man
Apple - Lobster
Apple - Boy

Orange - Clock
Orange - Telephone
Orange - Television
Orange - Spider
Orange - Man
Orange - Lobster
Orange - Boy

Clock - Telephone
Clock - Television
Clock - Spider
Clock - Man
Clock - Lobster
Clock - Boy

Telephone - Television
Telephone - Spider
Telephone - Man
Telephone - Lobster
Telephone - Boy

Television - Spider
Television - Man
Television - Lobster
Television - Boy

Spider - Man
Spider - Lobster
Spider - Boy

Man - Lobster
Man - Boy

Lobster - Boy
"""

' \n###Classes###         \nApple\nOrange\nClock\nTelephone\nTelevision\nSpider\nMan\nLobster\nBoy\n\n###Semantic Combinations###\nApple - Orange\nApple - Clock\nApple - Telephone\nApple - Television\nApple - Spider\nApple - Man\nApple - Lobster\nApple - Boy\n\nOrange - Clock\nOrange - Telephone\nOrange - Television\nOrange - Spider\nOrange - Man\nOrange - Lobster\nOrange - Boy\n\nClock - Telephone\nClock - Television\nClock - Spider\nClock - Man\nClock - Lobster\nClock - Boy\n\nTelephone - Television\nTelephone - Spider\nTelephone - Man\nTelephone - Lobster\nTelephone - Boy\n\nTelevision - Spider\nTelevision - Man\nTelevision - Lobster\nTelevision - Boy\n\nSpider - Man\nSpider - Lobster\nSpider - Boy\n\nMan - Lobster\nMan - Boy\n\nLobster - Boy\n'

**Import Gensim to access Word2Vec (300 Dimension)**

In [ ]:
import gensim.downloader as api
from scipy.spatial import distance
wv = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [ ]:
# Returns the corresponding word vector
def retVec(word):
  return wv.get_vector(word)

# Returs euclidean distances between two word vectors
def eucdist(word1,word2):
  return 1 - distance.cosine(retVec(word1), retVec(word2))

# Euc distances before adding extra feats.
print(eucdist('Apple','Orange'))
print(eucdist('Apple','Clock'))
print(eucdist('Apple','Phone'))
print(eucdist('Apple','TV'))
print(eucdist('Apple','Spider'))
print(eucdist('Apple','Man'))
print(eucdist('Apple','Lobster'))
print(eucdist('Apple','Boy'))


print(eucdist('Orange','Clock'))
print(eucdist('Orange','Phone'))
print(eucdist('Orange','TV'))
print(eucdist('Orange','Spider'))
print(eucdist('Orange','Man'))
print(eucdist('Orange','Lobster'))
print(eucdist('Orange','Boy'))


print(eucdist('Clock','Phone'))
print(eucdist('Clock','TV'))
print(eucdist('Clock','Spider'))
print(eucdist('Clock','Man'))
print(eucdist('Clock','Lobster'))
print(eucdist('Clock','Boy'))


print(eucdist('Phone','TV'))
print(eucdist('Phone','Spider'))
print(eucdist('Phone','Man'))
print(eucdist('Phone','Lobster'))
print(eucdist('Phone','Boy'))

print(eucdist('TV','Spider'))
print(eucdist('TV','Man'))
print(eucdist('TV','Lobster'))
print(eucdist('TV','Boy'))

print(eucdist('Spider','Man'))
print(eucdist('Spider','Lobster'))
print(eucdist('Spider','Boy'))

print(eucdist('Man','Lobster'))
print(eucdist('Man','Boy'))

print(eucdist('Lobster','Boy'))


0.25038832426071167
0.12709766626358032
0.23946939408779144
0.170773446559906
0.1510646939277649
0.11871977150440216
0.19865894317626953
0.15101148188114166
0.02602888084948063
0.2194230556488037
0.09528092294931412
0.10419514030218124
0.08557476848363876
0.09087955206632614
0.08176237344741821
0.2839314937591553
0.17443840205669403
0.1942809671163559
0.2289055436849594
0.23252813518047333
0.24624952673912048
0.227382093667984
0.17863905429840088
0.09509647637605667
0.17876292765140533
0.2113427370786667
0.04399801790714264
0.14182573556900024
0.04525585100054741
0.164581760764122
0.2647304832935333
0.22915314137935638
0.2316076159477234
0.16104820370674133
0.5896925330162048
0.21571102738380432


**Utility functions to add mean RGB-Euclidean Distances and Mean Edge Distances to word vectors**

In [ ]:
# For Apple
def makeVecApple(name):
  vec = retVec(name)
  vec = np.append(vec,[0.6989133827814864]) 
  vec = np.append(vec,[0.8266987351097107])
  vec = np.append(vec,[0.7173727345903107])
  vec = np.append(vec,[0.7304042567834301])
  vec = np.append(vec,[0.613490355694475])
  vec = np.append(vec,[0.1697301241362723])
  vec = np.append(vec,[0.13803807024906073])
  return vec
  
apple_vec = makeVecApple('Apple')
# apple_vec_fruit = makeVecApple('Fruit')
# apple_vec_device = makeVecApple('Device')

In [ ]:
# For Orange
def makeVecOrange(name):
  vec = retVec(name)
  vec = np.append(vec,[0.6942960684706365]) 
  vec = np.append(vec,[0.8371105519509027])
  vec = np.append(vec,[0.8468032335204911])
  vec = np.append(vec,[0.6496822025772303])
  vec = np.append(vec,[0.6891820237762419])
  vec = np.append(vec,[0.15074805196534014])
  vec = np.append(vec,[0.12871741698131164])
  return vec
orange_vec = makeVecOrange('Orange')
# orange_vec_fruit = makeVecOrange('Fruit')
# orange_vec_device = makeVecOrange('Device')

In [ ]:
# For Clock
def makeVecClock(name):
  vec = retVec(name)
  vec = np.append(vec,[0.6727336585235173]) 
  vec = np.append(vec,[0.7186481415883884])
  vec = np.append(vec,[0.7223022279553013])
  vec = np.append(vec,[0.6776505456644161])
  vec = np.append(vec,[0.5793402347984551])
  vec = np.append(vec,[0.1342179495721808])
  vec = np.append(vec,[0.1309685777489577])
  return vec

clock_vec = makeVecClock('Clock')
# clock_vec_fruit = makeVecClock('Fruit')
# clock_vec_device = makeVecClock('Device')

In [ ]:
# For Phone. Telephone is termed as Phone since it is more common.
def makeVecPhone(name):
  vec = retVec(name)
  vec = np.append(vec,[0.7743407077695059]) 
  vec = np.append(vec,[0.7762534786735532])
  vec = np.append(vec,[0.7360643837072662])
  vec = np.append(vec,[0.8021136449350464])
  vec = np.append(vec,[0.6394740131878117])
  vec = np.append(vec,[0.1252142658949014])
  vec = np.append(vec,[0.10931388571428688])
  return vec

phone_vec = makeVecPhone('Phone')
# phone_vec_fruit = makeVecPhone('Fruit')
# phone_vec_device = makeVecPhone('Device')

In [ ]:
# For TV. Television is termed as TV since it is more common.
def makeVecTV(name):
  vec = retVec(name)
  vec = np.append(vec,[0.746230650513263]) 
  vec = np.append(vec,[0.7726771561401076])
  vec = np.append(vec,[0.7902494348266198])
  vec = np.append(vec,[0.7777994723566739])
  vec = np.append(vec,[0.5785474506755405])
  vec = np.append(vec,[0.13073433643669852])
  vec = np.append(vec,[0.13547626302709853])
  return vec

tv_vec = makeVecTV('TV')
# tv_vec_fruit = makeVecTV('Fruit')
# tv_vec_device = makeVecTV('Device')

In [ ]:
# For Spider. Television is termed as TV since it is more common.
def makeVecSpider(name):
  vec = retVec(name)
  vec = np.append(vec,[0.6964503830214861]) 
  vec = np.append(vec,[0.7552778767676497])
  vec = np.append(vec,[0.7641662478004911])
  vec = np.append(vec,[0.7070006184950871])
  vec = np.append(vec,[0.5760483749193088])
  vec = np.append(vec,[0.08940934134122122])
  vec = np.append(vec,[0.08739553210847933])
  return vec

spider_vec = makeVecSpider('Spider')
# tv_vec_fruit = makeVecTV('Fruit')
# tv_vec_device = makeVecTV('Device')

In [ ]:
# For Man. Television is termed as TV since it is more common.
def makeVecMan(name):
  vec = retVec(name)
  vec = np.append(vec,[0.8196891792048556]) 
  vec = np.append(vec,[0.8256884853189225])
  vec = np.append(vec,[0.7775292910644415])
  vec = np.append(vec,[0.7500804947838658])
  vec = np.append(vec,[0.6073078032400057])
  vec = np.append(vec,[0.11704146315031076 ])
  vec = np.append(vec,[0.11760031046369603])
  return vec

man_vec = makeVecMan('Man')
# tv_vec_fruit = makeVecTV('Fruit')
# tv_vec_device = makeVecTV('Device')

In [ ]:
# For Lobster. Television is termed as TV since it is more common.
def makeVecLobster(name):
  vec = retVec(name)
  vec = np.append(vec,[0.5972181370511529]) 
  vec = np.append(vec,[0.8104048977976188])
  vec = np.append(vec,[0.7904165123284206])
  vec = np.append(vec,[0.6326307424533263])
  vec = np.append(vec,[0.6012651982773768])
  vec = np.append(vec,[0.14132818912360243])
  vec = np.append(vec,[0.12979228441186438])
  return vec

lobster_vec = makeVecLobster('Lobster')
# tv_vec_fruit = makeVecTV('Fruit')
# tv_vec_device = makeVecTV('Device')

In [ ]:
# For Boy. Television is termed as TV since it is more common.
def makeVecBoy(name):
  vec = retVec(name)
  vec = np.append(vec,[0.7807761740153684]) 
  vec = np.append(vec,[0.8135194248151673])
  vec = np.append(vec,[0.7941322520904851])
  vec = np.append(vec,[0.790675549283638])
  vec = np.append(vec,[0.6328791698158537])
  vec = np.append(vec,[0.12416545127769356 ])
  vec = np.append(vec,[0.12775947053774198])
  return vec

boy_vec = makeVecBoy('Boy')
# tv_vec_fruit = makeVecTV('Fruit')
# tv_vec_device = makeVecTV('Device')

In [ ]:
# Refined euclidean distance of two word vectors
def eucdist(wv1,wv2):
  return 1 - distance.cosine(wv1, wv2)

In [ ]:
pd.DataFrame({'Apple':apple_vec,
 'Orange':orange_vec,
 'Clock':clock_vec,
 'Phone':phone_vec,
 'TV':tv_vec,
 'Spider':spider_vec,
 'Man':man_vec,
 'Lobster':lobster_vec,
 'Boy':boy_vec}).to_csv('Added_feat_sem_vec.csv',index = False)

In [ ]:
print(eucdist(apple_vec,orange_vec))
print(eucdist(apple_vec,clock_vec))
print(eucdist(apple_vec,phone_vec))
print(eucdist(apple_vec,tv_vec))
print(eucdist(apple_vec,spider_vec))
print(eucdist(apple_vec,man_vec))
print(eucdist(apple_vec,lobster_vec))
print(eucdist(apple_vec,boy_vec))


print(eucdist(orange_vec,clock_vec))
print(eucdist(orange_vec,phone_vec))
print(eucdist(orange_vec,tv_vec))
print(eucdist(orange_vec,spider_vec))
print(eucdist(orange_vec,man_vec))
print(eucdist(orange_vec,lobster_vec))
print(eucdist(orange_vec,boy_vec))


print(eucdist(clock_vec,phone_vec))
print(eucdist(clock_vec,tv_vec))
print(eucdist(clock_vec,spider_vec))
print(eucdist(clock_vec,man_vec))
print(eucdist(clock_vec,lobster_vec))
print(eucdist(clock_vec,boy_vec))


print(eucdist(phone_vec,tv_vec))
print(eucdist(phone_vec,spider_vec))
print(eucdist(phone_vec,man_vec))
print(eucdist(phone_vec,lobster_vec))
print(eucdist(phone_vec,boy_vec))

print(eucdist(tv_vec,spider_vec))
print(eucdist(tv_vec,man_vec))
print(eucdist(tv_vec,lobster_vec))
print(eucdist(tv_vec,boy_vec))

print(eucdist(spider_vec,man_vec))
print(eucdist(spider_vec,lobster_vec))
print(eucdist(spider_vec,boy_vec))

print(eucdist(man_vec,lobster_vec))
print(eucdist(man_vec,boy_vec))

print(eucdist(lobster_vec,boy_vec))


# print(eucdist(apple_vec,apple_vec_fruit), eucdist(apple_vec,apple_vec_device))
# print(eucdist(orange_vec,orange_vec_fruit), eucdist(orange_vec,orange_vec_device))
# print(eucdist(clock_vec,clock_vec_fruit), eucdist(clock_vec,clock_vec_device))
# print(eucdist(phone_vec,phone_vec_fruit), eucdist(phone_vec,phone_vec_device))
# print(eucdist(tv_vec,tv_vec_fruit), eucdist(tv_vec,tv_vec_device))

0.41793270584989994
0.2986763368377141
0.398982492965532
0.38148238413001057
0.33967563745845086
0.3100461786219231
0.34521716600325847
0.343118578038333
0.2296840374545528
0.39238709080371525
0.3401388460818917
0.31605851055333045
0.29623440917571164
0.26885482279902295
0.3023757202888011
0.4239356498259865
0.3695270454864661
0.3611918839919932
0.3848368022922918
0.36365265156829274
0.4049065151689062
0.4226939556072824
0.360571511406812
0.2910376470744396
0.3274490769297247
0.3892632882593692
0.302177381236698
0.3674521140422389
0.2565455830755612
0.3938946464723605
0.4334385856518702
0.3772787111143592
0.41548093183407087
0.3184780677548953
0.6853337472635901
0.36860486879887544


In [ ]:
'''
Content of the CIFAR 100 Dataset
0: apple

1: aquarium_fish

2: baby

3: bear

4: beaver

5: bed

6: bee

7: beetle

8: bicycle

9: bottle

10: bowl

11: boy

12: bridge

13: bus

14: butterfly

15: camel

16: can

17: castle

18: caterpillar

19: cattle

20: chair

21: chimpanzee

22: clock

23: cloud

24: cockroach

25: couch

26: cra

27: crocodile

28: cup

29: dinosaur

30: dolphin

31: elephant

32: flatfish

33: forest

34: fox

35: girl

36: hamster

37: house

38: kangaroo

39: keyboard

40: lamp

41: lawn_mower

42: leopard

43: lion

44: lizard

45: lobster

46: man

47: maple_tree

48: motorcycle

49: mountain

50: mouse

51: mushroom

52: oak_tree

53: orange

54: orchid

55: otter

56: palm_tree

57: pear

58: pickup_truck

59: pine_tree

60: plain

61: plate

62: poppy

63: porcupine

64: possum

65: rabbit

66: raccoon

67: ray

68: road

69: rocket

70: rose

71: sea

72: seal

73: shark

74: shrew

75: skunk

76: skyscraper

77: snail

78: snake

79: spider

80: squirrel

81: streetcar

82: sunflower

83: sweet_pepper

84: table

85: tank

86: telephone

87: television

88: tiger

89: tractor

90: train

91: trout

92: tulip

93: turtle

94: wardrobe

95: whale

96: willow_tree

97: wolf

98: woman

99: worm
'''